<a href="https://colab.research.google.com/github/priyashah23/dissertation-reddit-controversy/blob/main/finetune_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is based off the HuggingFace article that gave a guide to how to fine tune a model.
The original code can be found here:
https://huggingface.co/learn/nlp-course/en/chapter7/3


In [ ]:
!pip install transformers[torch]
!pip install datasets

In [ ]:
import numpy as np
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from tqdm.notebook import tqdm
from google.colab import drive
from google.colab import userdata

In [ ]:
from huggingface_hub import notebook_login
notebook_login(userdata.get('token'))

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Finetune model using sentiment-analysis and then what?
MODEL_NAME = f"cardiffnlp/twitter-roberta-base-2022-154m"
model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

## Fine-tuning roBERTa model with pre-exisitng reddit sentiment data

Model Obtained from
https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
tokenizer.model_max_length

512

Reddit Gaming Data Came From
 https://www.kaggle.com/datasets/sainitishmitta04/23k-reddit-gaming-comments-with-sentiments-dataset/

In [ ]:
df = pd.read_csv("gdrive/MyDrive/Colab Notebooks/Dissertation/cleaned_subreddit_gaming.csv")
df

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281631 entries, 0 to 281630
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             281630 non-null  object 
 1   body                   281631 non-null  object 
 2   score_hidden           281628 non-null  object 
 3   created_utc_x          281628 non-null  object 
 4   score_x                281628 non-null  float64
 5   controversiality       281628 non-null  float64
 6   gilded_x               281628 non-null  float64
 7   is_moderator_comments  281628 non-null  object 
 8   created_utc_y          281628 non-null  object 
 9   domain                 281628 non-null  object 
 10  num_comments           281628 non-null  float64
 11  score_y                281628 non-null  object 
 12  title                  281628 non-null  object 
 13  gilded_y               281628 non-null  object 
 14  stickied               281628 non-nu

In [ ]:
data = df[~df.isna().any(axis=1)]

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 281625 entries, 0 to 281630
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             281625 non-null  object 
 1   body                   281625 non-null  object 
 2   score_hidden           281625 non-null  object 
 3   created_utc_x          281625 non-null  object 
 4   score_x                281625 non-null  float64
 5   controversiality       281625 non-null  float64
 6   gilded_x               281625 non-null  float64
 7   is_moderator_comments  281625 non-null  object 
 8   created_utc_y          281625 non-null  object 
 9   domain                 281625 non-null  object 
 10  num_comments           281625 non-null  float64
 11  score_y                281625 non-null  object 
 12  title                  281625 non-null  object 
 13  gilded_y               281625 non-null  object 
 14  stickied               281625 non-nu

In [ ]:
data.drop(229375, inplace=True)

<ipython-input-13-6a460ded2ea1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(229375, inplace=True)


In [ ]:
data = data[['body']]

In [ ]:
data.head()

,body
0,And yet we are still bombing the shit out of o...
1,Can’t deny that Nintendo products are built to...
2,I wonder if the owner survived.
3,This was at the Nintendo Store in NYC
4,can we bomb this shit out of this post before ...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 281624 entries, 0 to 281630
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   body    281624 non-null  object
dtypes: object(1)
memory usage: 4.3+ MB


In [ ]:
# Transform the pandas dataset into a dataset (7k, 5k, 3k)
dataset = Dataset.from_pandas(data).train_test_split(test_size=0.25, shuffle = True, seed = 200)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['body', '__index_level_0__'],
        num_rows: 211218
    })
    test: Dataset({
        features: ['body', '__index_level_0__'],
        num_rows: 70406
    })
})

In [ ]:
def tokenize_function(examples):
  result = tokenizer(examples['body'])
  if tokenizer.is_fast:
    result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
  return result

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["body",  "__index_level_0__"])
tokenized_dataset

Map:   0%|          | 0/211218 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (539 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/70406 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 211218
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 70406
    })
})

In [ ]:
chunk_size = 256

def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
# Remove uncessarsary columns
lm_dataset = tokenized_dataset.map(group_texts, batched=True)

Map:   0%|          | 0/211218 [00:00<?, ? examples/s]

Map:   0%|          | 0/70406 [00:00<?, ? examples/s]

In [ ]:
lm_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 20428
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 6807
    })
})

In [ ]:
tokenizer.decode(lm_dataset["train"][1]["input_ids"])


". Agghhhh... So many LPs just because of this.</s><s>I loved gaming with my daughter! For the first few months I got through loads of games and box sets... the Switch’s split controller was particularly useful.  \r\n\r\nShe’s now very clingy at 17 months. Cries if I hold her. Cries if I don’t. It’s both cute and frustrating. The most surprising thing about being a dad is how quickly my emotions change from rage to love.</s><s>DOG PUSHES IT TO THE LIMIT</s><s>ITS A GAMEBOY NO MATTER WHAT, GET IT RIGHT</s><s>I’ve been replaying mass effect and I’m going to fight the thorian and his thralls just keep running up at the squad NPCs and just obliterating them with some poison breath attack. I always have to run back and do a spray and pray on them. Crude...but effective none the less. </s><s>I'm down!!!!</s><s>Hopefully they redo the crafting/mod system, redo or severely cut down on settlements, survival mode from the get go, real guns ( like in the original fallouts and tactics, 10mm"

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
from transformers import TrainingArguments
import torch

batch_size = 32
# Show the training loss with every epoch
logging_steps = len(lm_dataset["train"]) // batch_size
model_name = MODEL_NAME.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-reddit",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.247100,2.022149
2,2.104600,1.985541
3,2.065800,1.971727


TrainOutput(global_step=1917, training_loss=2.139196769981006, metrics={'train_runtime': 350.9713, 'train_samples_per_second': 174.613, 'train_steps_per_second': 5.462, 'total_flos': 8066980326316032.0, 'train_loss': 2.139196769981006, 'epoch': 3.0})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}"

'>>> Perplexity: 7.17'

In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1710602939.3fb9cb5cd671.345.1:   0%|          | 0.00/297 [00:00<?, ?B/s]

events.out.tfevents.1710602566.3fb9cb5cd671.345.0:   0%|          | 0.00/6.51k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Priya-Shah23/twitter-roberta-base-2022-154m-finetuned-reddit/commit/d852ffd5910b1e32b66174d5a7f687938f9546c3', commit_message='End of training', commit_description='', oid='d852ffd5910b1e32b66174d5a7f687938f9546c3', pr_url=None, pr_revision=None, pr_num=None)

## Get Hidden Layer from fine-tuned model

In [ ]:
trainer.save_model("gdrive/MyDrive/Colab Notebooks/Dissertation/reddit-roberta-ft")

In [ ]:
def obtain_hidden_layer(text: str):
  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
  outputs = amazing(**inputs)
  print(outputs.hidden_layer)

### Prepare Dataset

In [ ]:
# Get text from google drive
df = pd.read_csv('gdrive/MyDrive/Colab Notebooks/Dissertation/post_with_variance.csv')
comments_df = pd.read_csv('gdrive/MyDrive/Colab Notebooks/Dissertation/reddit_comments.csv')

In [ ]:
def clean_sentence(sentence):
  # Remove any hyper links
  sentence = re.sub("https?:\/\/t.co\/[A-Za-z0-9]*", '', sentence)
  # Remove white space
  sentence = sentence.replace('\r', '').replace('\n', '')
  return sentence

In [ ]:
post_comments = []
for post_id in tqdm(df["submission_id"]):
  list_of_comments = []
  post_id = 't3_' + post_id

  # Get comments from that post:
  comments = comments_df.query('@post_id == comment_link_id')
  for comment in comments['comment_body']:
    # Skip over delete comments
    if comment == "[deleted]":
      continue
    comment = clean_sentence(comment)
    list_of_comments.append(comment)
  post_comments.append(list_of_comments)

df["comments"] = post_comments

In [ ]:
combined_df = df[["submission_title", "comments", "is_controversial"]]
combined_df.head()

Get Embeddings

In [ ]:
train_ratio = 0.80
test_ratio = 0.20

# train test split
dataset_test_train = Dataset.from_pandas(combined_df).train_test_split(test_size=test_ratio, shuffle = True, seed = 200)

In [ ]:
train_embeddings = get_embeddings(dataset_test_train["train"])